In [62]:
import scanpy as sc
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
import itertools as it

In [64]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"

In [67]:
# cells = np.array([
#     [1, 0, 0, 0, 0],
#     [1, 1, 0, 0, 0],
#     [0, 0, 1, 0, 0],
#     [0, 0, 1, 1, 0],
#     [0, 0, 0, 0, 1],
# ])
# ncells = len(cells)

adata = sc.read_10x_h5(FILE)
X = adata.X.todense()
def cell(i):
    return np.array(X[i, :]).flatten()
ncells = X.shape[0]
cells = []
for i in tqdm.trange(ncells):
    cells.append(cell(i))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
100%|███████████████████████████████████| 5697/5697 [00:00<00:00, 273222.53it/s]


In [99]:
nonzeros = [np.count_nonzero(i) for i in cells]
def similarity(cell1, cell2):
    return nonzeros[cell1] - np.count_nonzero(cells[cell1] - cells[cell2])

def best_friend(cell):
    return max(it.chain(range(cell), range(cell + 1, ncells)), key=lambda other: similarity(cell, other))

def max_similarity(cell):
    return max(map(lambda other: similarity(cell, other), it.chain(range(cell), range(cell + 1, ncells))))

In [119]:
def simmify(array):
    friends = []
    simmed = []
    for i, row in tqdm.tqdm(enumerate(array)):
        if i + 1 in friends or max_similarity(i) <= 0:
            friends.append(0)
            simmed.append(row)
        else:
            f = best_friend(i)
            friends.append(f + 1)
            simmed.append(row - array[f])
    return friends, np.array(simmed)

In [120]:
def unsimmify(friends, simmed):
    array = []
    for friend, row in zip(friends, simmed):
        if not friend:
            array.append(row)
        else:
            array.append(row + simmed[friend - 1])
    return np.array(array)

In [121]:
len(cells)

5697

In [122]:
a, b = simmify(cells)

5697it [02:29, 37.99it/s]


In [97]:
best_friend(0)

4399

In [115]:
similarity(2, 4933)

10

In [116]:
b[2]

array([ 5.,  0., 13., ...,  1.,  0.,  0.], dtype=float32)

In [127]:
all(cells[2] == (b[2] + cells[4933]))

True

In [129]:
for i in range(ncells):
    if a[i] and not all(cells[i] == (b[i] + b[a[i] - 1])):
        print(i)
        input()

218


KeyboardInterrupt: Interrupted by user

In [132]:
b[218] + b[10]

array([ 5.,  0., 28., ...,  2.,  0.,  0.], dtype=float32)

In [131]:
a[218]

11

In [126]:
a

[0,
 0,
 4934,
 3326,
 1595,
 3594,
 0,
 0,
 311,
 2285,
 1343,
 1054,
 0,
 5180,
 4100,
 0,
 5515,
 5155,
 4033,
 3016,
 5474,
 1199,
 0,
 3556,
 0,
 0,
 3282,
 4158,
 0,
 0,
 2245,
 1054,
 1607,
 3594,
 5078,
 4414,
 0,
 579,
 1867,
 4356,
 3193,
 5173,
 3236,
 1605,
 3151,
 4920,
 0,
 0,
 3678,
 359,
 3744,
 419,
 912,
 0,
 1819,
 3681,
 1711,
 1829,
 931,
 2160,
 0,
 4030,
 0,
 0,
 3053,
 0,
 1281,
 4505,
 4061,
 112,
 2662,
 0,
 5129,
 1940,
 4477,
 2195,
 3431,
 4414,
 4864,
 4909,
 844,
 1183,
 0,
 1026,
 0,
 840,
 4348,
 0,
 5245,
 3285,
 5615,
 3108,
 0,
 0,
 1183,
 170,
 0,
 4018,
 1199,
 837,
 2712,
 2704,
 5609,
 419,
 1492,
 1399,
 0,
 1565,
 3442,
 5571,
 3236,
 0,
 116,
 0,
 3385,
 0,
 4634,
 5360,
 442,
 2286,
 0,
 5360,
 0,
 0,
 0,
 384,
 3664,
 3303,
 3155,
 579,
 1335,
 3129,
 5399,
 928,
 4646,
 3594,
 4312,
 2633,
 0,
 732,
 3885,
 3384,
 461,
 637,
 1607,
 4742,
 4110,
 0,
 0,
 4745,
 25,
 2040,
 0,
 5569,
 1829,
 749,
 0,
 1054,
 3326,
 1790,
 406,
 1548,
 814,
 

In [124]:
np.count_nonzero(b)

1807372

In [136]:
(1 - 1807372 / 1850701) * 100

2.3412209751872415